# Introduction to RNNs: learning the alphabet

A simple example to understand how Recurrent Neural Networks learn sequential patterns.

## Key concepts

- **Sequence learning**: RNNs process input one step at a time, maintaining a "memory" (hidden state) of what they've seen.
- **Next-character prediction**: Given a sequence like `['a', 'b', 'c']`, predict the next character (`'d'`).
- **Hidden state**: The internal memory that carries information from previous timesteps (or indexes) in a sequence.

## How the RNN processes a sequence

Given input `['a', 'b', 'c']`, the RNN predicts `'d'`:

```text

                     ┌─◀──────┐ 'h₁', then 'h₂'
                     │        ▲ 
                     ▼        |
                    ┌──────────┐
                    |   RNN    |         ┌───────┐
'a', then 'b', ────▶| (hidden  |──'h₃'──▶│ Dense │───▶ 'c'
   then 'c'         | state h) |         └───────┘ 
                    └──────────┘


h₁ = RNN('a', h₀)        # h₁ "knows" about 'a'
h₂ = RNN('b', h₁)        # h₂ "knows" about 'a', 'b'  
h₃ = RNN('c', h₂)        # h₃ "knows" about 'a', 'b', 'c'
output = Dense(h₃) → 'd' # Final hidden state predicts next char
```

The same RNN weights are used at each step within a sequence — only the hidden state changes.

## Setup

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

### Configuration

In [ ]:
epochs=300
batch_size=4

# Force CPU only
tf.config.set_visible_devices([], 'GPU')

## 1. Data preparation

### 1.1. Create the alphabet dataset

In [ ]:
# Create alphabet mapping
alphabet = 'abcdefghijklmnopqrstuvwxyz'
char_to_idx = {c: i for i, c in enumerate(alphabet)}
idx_to_char = {i: c for i, c in enumerate(alphabet)}
vocab_size = len(alphabet)

print(f'Vocabulary: {alphabet}')

print('\nMappings:\n')

for key, val in char_to_idx.items():
    print(f' {key} -> {val}')

### 1.2. Training 'freatures' & labels

In [ ]:
seq_length = 3

y = alphabet[seq_length:] + 'a'  # Wrap around: 'xyz' -> 'a'
x = list(zip(*[alphabet[i:] for i in range(seq_length)]))

print('\nInput sequence  -> label')

for features, label in zip(x, y):
    print(f'{features} -> {label}')

print(f'X shape: {np.array(x).shape}, y shape: {np.array(list(y)).shape}')

### 1.3. One-hot encoding

In [ ]:
X_indices = np.array([[char_to_idx[c] for c in seq] for seq in x])
y_indices = np.array([char_to_idx[c] for c in y])

X_onehot = to_categorical(X_indices, num_classes=vocab_size)
y_onehot = to_categorical(y_indices, num_classes=vocab_size)

print('First sequence (one-hot):')

for ele in X_onehot[0][:, :10]:
    print(f' {(", ").join(map(str, ele))}...')

print('\nFirst label (one-hot):')
print(f' {(", ").join(map(str, y_onehot[0][:10]))}...')

print(f'\nX shape (one-hot): {X_onehot.shape}')
print(f'y shape (one-hot): {y_onehot.shape}')

## 2. Recurrent model

A simple architecture:
1. **Input**: One-hot encoded characters (26-dimensional vectors)
2. **SimpleRNN**: Process the sequence, output final hidden state  
3. **Dense**: Predict the next character

With one-hot encoding, each input character is a vector of 26 zeros with a single 1 at the character's index position.

### 2.1. Define RNN

In [ ]:
# Model parameters
hidden_dim = 8 # RNN hidden state size

model = Sequential([
    SimpleRNN(hidden_dim, input_shape=(seq_length, vocab_size)),
    Dense(vocab_size, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

### 2.2. Train RNN

In [ ]:
history = model.fit(
    X_onehot, y_onehot,
    epochs=epochs,
    batch_size=batch_size,
    verbose=0
)

### 2.3. Learning curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6, 3))

axes[0].plot(history.history['loss'])
axes[0].set_title('Training loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')

axes[1].plot(history.history['accuracy'])
axes[1].set_title('Training accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')

plt.tight_layout()
plt.show()

## 3. Model evaluation

In [ ]:
def predict_next(chars, model):
    '''Predict the next character given a sequence.'''

    indices = np.array([[char_to_idx[c] for c in chars]])
    onehot = to_categorical(indices, num_classes=vocab_size)
    pred = model.predict(onehot, verbose=0)
    next_idx = np.argmax(pred)

    return idx_to_char[next_idx]

In [ ]:
# Test predictions
test_sequences = ['abc', 'def', 'mno', 'stu', 'wxy']

print('Predictions:')

for seq in test_sequences:
    if len(seq) == seq_length and all(c in char_to_idx for c in seq):
        pred = predict_next(seq, model)
        print(f' {seq} -> {pred}')